In [63]:
# Imports
import numpy as np
import random as random
import csv

In [ ]:
# Initializations and opening csv files
sigmoid = lambda y: 1/(1+np.exp(-y))
sigmoid_derivative = lambda x: x * (1-x)
train_data = []
test_data = []
epochs = 5

# Training data from csv
train_data_file = open('./train.csv')
train_data_rows = csv.reader(train_data_file)
next(train_data_rows, None)


test_data_file = open('./test.csv')
test_data_rows = csv.reader(test_data_file)
next(test_data_rows, None)

In [82]:
# Formatting train and test data
for row in train_data_rows:
    head = int(row[0])
    inputVals = list(map(lambda x: int(x)/255, row))
    inputVals = list(map(lambda x: [x], inputVals))
    outputVals = list(map(lambda x: [x], [0] * 10))
    outputVals[head] = [1]
    inputVals.pop(0)
    train_data.append((inputVals, outputVals))

for row in test_data_rows:
    inputVals = list(map(lambda x: int(x)/255, row))
    inputVals = list(map(lambda x: [x], inputVals))
    test_data.append(inputVals)


In [85]:
class NeuralNetwork:
  def __init__(self, numInput, numHidden, numOutput, learningRate):
    self.numInput = numInput
    self.numHidden = numHidden
    self.numOutput = numOutput
    self.learningRate = learningRate

    self.hidden_weights = np.random.uniform(-0.01, 0.01, (numHidden, numInput))
    self.output_weights = np.random.uniform(-0.01, 0.01, (numOutput, numHidden))
    self.hidden_bias = np.random.uniform(-0.01, 0.01, (numHidden, 1))
    self.output_bias = np.random.uniform(-0.01, 0.01, (numOutput, 1))

  def test(self, inputs): 
    # Generating the hidden layer outputs
    hidden_layer = np.dot(self.hidden_weights, inputs)
    hidden_layer += self.hidden_bias
    hidden_layer = sigmoid(hidden_layer)

    # Generating the outputs
    outputs = np.dot(self.output_weights, hidden_layer)
    outputs += self.output_bias
    outputs = sigmoid(outputs)

    return outputs

  def train(self, inputs, targets):
    # Generating the hidden layer outputs
    hidden_layer = np.dot(self.hidden_weights, inputs)
    hidden_layer += self.hidden_bias
    hidden_layer = sigmoid(hidden_layer)

    # Generating the outputs
    outputs = np.dot(self.output_weights, hidden_layer)
    outputs += self.output_bias
    outputs = sigmoid(outputs)

    # Calculating output errors
    output_errors = targets - outputs

    # Calculating gradient for output weights
    output_gradients = sigmoid_derivative(outputs)
    output_gradients = output_gradients * output_errors
    output_gradients = self.learningRate * output_gradients

    # Calculating delta output weights
    delta_output_weights = np.dot(output_gradients, hidden_layer.T)
    self.output_weights += delta_output_weights
    self.output_bias += output_gradients
  
    # Calculating hidden layer errors
    hidden_errors = np.dot(self.output_weights.T, output_errors)

    # Calculating gradient for hidden weights
    hidden_gradients = sigmoid_derivative(hidden_layer)
    hidden_gradients = hidden_gradients * hidden_errors
    hidden_gradients = self.learningRate * hidden_gradients

    # Calculating delta output weights
    delta_hidden_weights = np.dot(hidden_gradients, inputs.T)
    self.hidden_weights += delta_hidden_weights
    self.hidden_bias += hidden_gradients

In [86]:
model = NeuralNetwork(784,784,10,0.1)
for i in range(epochs):
  for d in train_data:
    inputs, targets = d
    model.train(np.array(inputs), np.array(targets))



In [87]:
with open('./test_results.csv', 'w') as test_results_file:
    test_results_file.truncate()
    writer = csv.writer(test_results_file)
    writer.writerow(['ImageId', 'Label'])

    idx = 1
    for inputs in test_data:
        res = model.test(inputs).tolist()
        res = [item for sublist in res for item in sublist]
        row = [str(idx), str(res.index(max(res)))]
        writer.writerow(row)
        idx+=1